In [1]:
from importlib import reload
if 'andor' in dir():
    reload(andor)
else:
    import andor
try:
    andor.initialize_ctypes()
except:
    raise RuntimeError("Cannot load shared libray, not installed?")
#
from andorCodes import AndorCodes
##
from ipywidgets import widgets
from IPython.display import clear_output
##
import asyncio
import concurrent.futures
import sys
import time
##
from matplotlib import use,pyplot,colors
use('agg')
import numpy

---
### Asyncio and widgets

In [2]:
## Initialise the asyncio events
#
readout=asyncio.Event() # when set, the readout can commence
abort=asyncio.Event() # when set, the readout can stop
future_imagedisplayer,future_producer,future_consumer=None,None,None
loop = asyncio.get_event_loop() # the Tornado loop
dataqueue = asyncio.Queue(loop=loop)
imagequeue = asyncio.Queue(loop=loop)
gather_future=None # to keep things nice

In [3]:
## Helpful functions
#
def abort_and_clear():
    try:
        andor.abort_acquisition()
    except:
        print("No acquisition to abort")
    finally:
        andor.free_internal_memory()
    return

def start_acq():
    global configured
    if not configured:
        print("Not configured")
        return
    try:
        andor.start_acquisition()
    except:
        print(f"Acquisition can't be started: {andor.last_ret_code}={AndorCodes.values[andor.last_ret_code]}")
    return


async def wait_for_ready():
    while andor.get_status()==AndorCodes.DRV_ACQUIRING:
        await asyncio.sleep(0.2)

def shutdown():
    if andor.initialized:
        abort_and_clear()
        try:
            andor.shutdown()
        except:
            print(f"Failed to shutdown because: {andor.last_ret_code}={AndorCodes.values[andor.last_ret_code]}")
        else:
            andor.initialized=False

def initialize():
    global readout,abort
    if not andor.initialized:
        try:
            andor.initialize_sdk()
        except:
            raise RuntimeError("Cannot initialise SDK, already initialised elsewhere?")
        andor.choose_camera(0) # first
        abort.clear()
        readout.clear()

def configure():
    global configured    
    if not andor.initialized:
        print("Andor not initialized")
        return
    ##
    andor.set_region(region)
    andor.set_exposure(exposure_time)
    print(f"Asked for exposure time is {1e3*exposure_time:.1f}ms")
    andor.set_shutter(shutter_mode,ttl_high,open_time,close_time)
    andor.set_trigger(trigger_mode)
    print(f"Tigger mode is '{trigger_mode}'")
    # No need to do this, the default it seems
    andor.set_acquisition(acq_mode)
    print(f"Acquisition mode is '{acq_mode}'")
    andor.set_read(read_mode)
    print(f"Read mode is '{read_mode}'")
    print(f"Binning HxV is {hbinning}x{vbinning}")
    print("ROI is (L,R,B,T)={0[0]}x{0[1]},{0[2]},{0[3]}".format(region_lrbt))
    times=andor.get_timing()
    print(f"Therefore, exposure time is {1e3*times[0]:.1f}ms and acquisition time is {1e3*times[1]:.1f}ms")
    img_capacity=andor.get_size_of_circular_buffer()
    print(f"Expect {times[1]*img_capacity:.2f}s before circular buffer is full")
    print(f"Shutter {'is' if andor.get_shutter_info() else 'is not'} available")
    print(f"Shutter set to TTL o/p {'high' if ttl_high else 'low'}")
    print(f"Shutter mode is '{shutter_mode}'")
    print(f"Shutter open/close time is {open_time}/{close_time}ms")
    configured=True

In [4]:
## Widgets
#
buttonLabels=("Start","Stop","Abort")
buttonEnabled=[ False ]*3
buttonStyle="success","warning","danger"
button1W,button2W,button3W=[
        widgets.Button(description=buttonLabels[i],disabled=buttonEnabled[i],button_style=buttonStyle[i])
        for i in (0,1,2)
    ]
acq_buttonsHBW=widgets.HBox(
        [button1W,button2W,button3W],
        layout=widgets.Layout(
            width='100%',
            border='solid 1px gray',
            margin='0px 10px 10px 0px',
            padding='5px 5px 5px 5px',)
    )

buttonLabels=("Shutdown","Init","Configure")
buttonEnabled=[False]*3
buttonStyle="warning","info",""
button4W,button5W,button6W=[
        widgets.Button(description=buttonLabels[i],disabled=buttonEnabled[i],button_style=buttonStyle[i])
        for i in (0,1,2)
    ]
ctrl_buttonsHBW=widgets.HBox(
        [button4W,button5W,button6W],
        layout=widgets.Layout(
            width='100%',
            border='solid 1px gray',
            margin='0px 10px 10px 0px',
            padding='5px 5px 5px 5px',)
    )
statusOpW=widgets.Output(
        disabled=False,
        layout=widgets.Layout(
            border='solid 1px blue',
            width='50%',height='400px')
    )
imageOpW=widgets.Output(
        disabled=False,
        layout=widgets.Layout(
            border='solid 1px red',
            margin='0px 10px 10px 0px',
            padding='5px 5px 5px 5px',
            width='50%',height='400px')
    )
opHBW=widgets.HBox([imageOpW,statusOpW],layout=widgets.Layout(width='100%'))
panelVBW=widgets.VBox([acq_buttonsHBW,opHBW,ctrl_buttonsHBW])


In [5]:
@statusOpW.capture(clear_output=True)
def startReadout(button):
#     print("start")
    start_acq()
#     print("\tr/o set")    
    readout.set()
@statusOpW.capture(clear_output=True)
def stopReadout(button):
    print("stop")
    abort_and_clear()
    print("\tr/o unset")    
    readout.clear()

@statusOpW.capture(clear_output=True)
def abortReadout(button):
    print("abort") 
    abort_and_clear()
    print("\tabort set")    
    abort.set()

@statusOpW.capture(clear_output=True)
def shutdownSDK(button):
    print("sdk shutdown") 
    try:
        shutdown()
    except:
        print("exception raised!")
    print("\tsdk init status: "+str(andor.initialized))
    
@statusOpW.capture(clear_output=True)
def initializeSDK(button):
    print("sdk initialize") 
    try:
        initialize()
    except:
        print("exception raised!")
    print("\tsdk init status: "+str(andor.initialized))
    
@statusOpW.capture(clear_output=True)
def confCam(button):
    print("configure") 
    try:
        configure()
    except:
        print("exception raised!")
        raise
    
button1W.on_click(startReadout)
button2W.on_click(stopReadout)
button3W.on_click(abortReadout)
button4W.on_click(shutdownSDK)
button5W.on_click(initializeSDK)
button6W.on_click(confCam)

In [11]:
## Coroutines
#
async def produce(queue):
    global readout,abort,loop
    with statusOpW:
        start_time=time.time()
        n_imgs=0
        while not abort.is_set():
#             print("<",end="")
            await readout.wait()
#            with concurrent.futures.ThreadPoolExecutor() as pool:
#                await loop.run_in_executor(None,andor.wait_for_acquisition)
            await loop.run_in_executor( None, andor.wait_for_acquisition )
            if andor.last_ret_code==AndorCodes.DRV_NO_NEW_DATA:
                abort.set()
                andor.abort_acquisition()
                print(f"Acquisition ended without data {AndorCodes.values[andor.last_ret_code]}\n")
                break           
            n_imgs+=1
            # TODO \/ capture exception here and re-raise if andor.last_ret_code!=AndorCodes.DRV_NO_NEW_DATA
            try:
                image_indices=andor.get_number_available_images()
            except AssertionError:
                if andor.last_ret_code==AndorCodes.DRV_NO_NEW_DATA:
                    print(".")
                    continue
                else:
                    raise # re-raise
            data,first,last=andor.get_images(image_indices[1],image_indices[1]) # just the last image
            data=data.reshape([1]+list(region.shape)) # one frame of 2D data
            delta_t=time.time()-start_time
            queue_entry=[image_indices,delta_t,data]
            queue.put_nowait(queue_entry) # could throw an exception!
#             print(">",end="")

    await wait_for_ready()
    await queue.put(None)
    print('<end>')


async def consume(queue,imagequeue):
    with statusOpW:
        f=0
        while True:
            # wait for an item from the producer
            payload = await queue.get()
            f+=1
            if payload is None:
                # the producer emits None to indicate that it is done
                break

            queue_entry=[f,payload]
            await imagequeue.put(queue_entry) # now shove onto another queue, can block

    print('{end}')

async def imagedisplay(iqueue):
    starttime=time.time()
    blank=numpy.zeros(region.shape)
    with imageOpW:
        fig=pyplot.figure(figsize=(5,5))
        ax=fig.subplots(1)
        img=ax.imshow(blank)
#        display(fig)
#        clear_output()
    while True: # this never exits
        item = await iqueue.get()
        # process the item
        f,payload=item
        indices,ftime,data=payload
        with imageOpW:
            clear_output(wait=True)
            img.set_array(data.sum(0))
            img.set_clim(img._A.min(),img._A.max())            
            ax.set_title(f"{f}/{indices[1]},({data.min()}->{data.max()})")
            display(fig)
        with statusOpW:
            clear_output()
            print(f"t={ftime:.2f}")
    print('{end}')
    


---
### Andor setup

In [7]:
## Initialise the SDK
#
initialize()
print("Initialised SDK")
print("Set current camera to be #0 (first)")
ccd_height, ccd_width=andor.detector_size()
print(f"Size of detector is {ccd_height}x{ccd_width}")
configured=False

Initialised SDK
Set current camera to be #0 (first)
Size of detector is 1024x1024


In [8]:
## detector configuration parameters
#
hbinning,vbinning=(1,1) # horizontal then vertical
region_lrbt=1,1024,1,1024#100,200,100,200
region=numpy.empty([1024,1024])
region=andor.Region(hbinning,vbinning,region_lrbt[0],region_lrbt[1],region_lrbt[2],region_lrbt[3])
exposure_time=0.05 # [s]
ttl_high=True
shutter_mode='Open'
open_time,close_time=0,0
trigger_mode='Internal'
acq_mode='Continuous'
read_mode='Image'

---

In [12]:
if gather_future is not None:
    try:
        gather_future.cancel() # if we were waiting because cell re-run, nix it
    except CancelledError:
        print("Cancellation exception raised")
    except:
        raise

readout.clear()
abort.clear()
display(panelVBW)
producer=produce(dataqueue)
consumer=consume(dataqueue,imagequeue)
displayer=imagedisplay(imagequeue)
gather_future=asyncio.gather(producer,consumer,displayer,loop=loop)

In [10]:
status=andor.get_status()
print(f"Status is: {status}={AndorCodes.values[status]}")
print(f"Readout state is {readout.is_set()}")
print(f"Abort state is {abort.is_set()}")
print(f"Data Q is empty? {dataqueue.empty()}")
print(f"Image Q is empty? {imagequeue.empty()}")

Status is: 20073=DRV_IDLE
Readout state is False
Abort state is False
Data Q is empty? True
Image Q is empty? True


---